In [1]:

from mmvae_hub.experiment_vis.utils import *
from mmvae_hub.utils.MongoDB import MongoDatabase

[16:00:39 CEST] Log file: /Users/Hendrik/src/MMVAE_Hub/mmvae_hub/logger/logs/UTC-20210928-140039.log
/Users/Hendrik/opt/anaconda3/envs/mmvae/lib/python3.9/site-packages/pytorch_lightning/metrics/__init__.py:43: LightningDeprecationWarning: `pytorch_lightning.metrics.*` module has been renamed to `torchmetrics.*` and split off to its own package (https://github.com/PyTorchLightning/metrics) since v1.3 and will be removed in v1.5
  rank_zero_deprecation(


cpu


In [2]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

In [3]:
b = {exp['_id'].split('_')[0] for exp in experiments.find({})}
print(b)

{'Mimic', 'polymnist', 'mnistsvhntext', 'MST'}


In [4]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  136 experiments in the database.


In [5]:
df = make_experiments_dataframe(experiments)
# df.style.set_properties(subset=['_id'], **{'width': '1px'})

/Users/Hendrik/opt/anaconda3/envs/mmvae/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/Hendrik/opt/anaconda3/envs/mmvae/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


skipping experiment polymnist_iwmopgfm__2021_09_26_21_13_00_860149


In [6]:
prd_cols = [col for col in df.columns if col.startswith('prd')]

In [7]:
show_cols = ['score','method', 'end_epoch', 'class_dim','max_beta','beta_warmup','num_mods', '_id', 'score_lr', 'score_gen','score_prd', 'version', 'expvis_url','score_lr_q0','score_lr_zk', *prd_cols]
iwshow_cols = ['score','method', 'K','qz_x' ,'end_epoch', 'class_dim','max_beta','beta_warmup','num_mods', '_id', 'score_lr', 'score_gen','score_prd', 'version', 'expvis_url','score_lr_q0','score_lr_zk', *prd_cols]
display_df(df[show_cols].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
39,6.453927,mopoe_4,149,640.0,2.0,50.0,3.0,MST_mopoe_2021_09_10_21_36_19_019375,0.926845,0.762611,0.383157,0.0.22-dev,NaN,0.926845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.675000,0.015333,0.679465,0.477932,0.666566,0.450311,0.672088,0.002097,0.520672,0.297634,0.259556,0.495300,0.182475,0.464360,0.260525,0.011198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,6.370193,mopoe_4,299,1280.0,2.0,50.0,NaN,mnistsvhntext_mopoe_2021_09_23_10_57_27_193406,0.928698,0.752893,0.376303,0.0.27-dev,https://ppb.hendrikklug.xyz/816d1a565cd3.pdf,0.928698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615525,0.042754,0.617729,0.388764,0.621300,0.370225,0.631144,0.022558,0.461493,0.252462,0.409895,0.405539,0.329378,0.418549,0.411885,0.021654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5.568942,iwmogfm_4_0,299,512.0,0.0,500.0,3.0,polymnist_iwmogfm_2021_08_30_12_11_26_824957,0.922457,0.796820,0.289024,0.0.22-dev,https://ppb.hendrikklug.xyz/5c2a6676ea08.pdf,0.922457,NaN,0.547333,0.074786,0.540079,0.561758,0.140831,0.532492,0.535866,0.446089,0.061408,0.533035,0.138123,0.456854,0.057423,0.576795,0.068345,0.042195,0.564898,0.447889,0.048674,0.097341,0.444598,0.018003,0.000824,0.000928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [8]:
a = {i.split('_')[0] for i in  df['_id'].to_list()}
print(a)

{'Mimic', 'polymnist', 'mnistsvhntext', 'MST'}


In [9]:
df.loc[df['_id'] == 'polymnist_mofop_2021_07_15_19_41_22_973638', 'num_flows']

Series([], Name: num_flows, dtype: float64)

# Runs on mimic

In [10]:
mimic_ids = [(exp['_id'], exp['flags']['end_epoch']) for exp in experiments.find({}) if exp['_id'].startswith('Mimic')]
mimic_ids

[('Mimic_mopoe_2021_09_20_09_13_55_749457', 150),
 ('Mimic_mopoe_2021_09_20_09_16_50_591785', 150)]

In [11]:
display_df(df.loc[df['_id'].str.startswith('Mimic')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
76,1.459602,mopoe_4,149,640.0,1.0,100.0,3.0,Mimic_mopoe_2021_09_20_09_13_55_749457,0.922130,0.126112,0.00008,0.0.25-dev,https://ppb.hendrikklug.xyz/f4642369640d.pdf,0.922130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000827,0.0,0.0,0.0,0.000456,0.0,0.0,0.0,0.0,0.0,0.0
78,1.451198,mopoe_4,149,1280.0,1.0,100.0,3.0,Mimic_mopoe_2021_09_20_09_16_50_591785,0.934722,0.112696,0.00000,0.0.25-dev,https://ppb.hendrikklug.xyz/b4129aaa5279.pdf,0.934722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


# Runs on mnistsvhntext

In [12]:
mnistsvhntext_ids = [(exp['_id'], exp['flags']['end_epoch']) for exp in experiments.find({}) if exp['_id'].startswith('mnistsvhntext')]
mnistsvhntext_ids

[('mnistsvhntext_mopoe_2021_09_23_10_57_27_193406', 300)]

In [13]:
display_df(df.loc[df['_id'].str.startswith('mnistsvhntext')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
80,6.370193,mopoe_4,299,1280.0,2.0,50.0,NaN,mnistsvhntext_mopoe_2021_09_23_10_57_27_193406,0.928698,0.752893,0.376303,0.0.27-dev,https://ppb.hendrikklug.xyz/816d1a565cd3.pdf,0.928698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615525,0.042754,0.617729,0.388764,0.6213,0.370225,0.631144,0.022558,0.461493,0.252462,0.409895,0.405539,0.329378,0.418549,0.411885,0.021654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Method: iwMoGfM
Mixture of Parameter Generalized f-means

In [14]:
display_df(df.loc[df['method'].str.startswith('iwmogfm_')][[*show_cols]].sort_values(by=['score'], ascending=False).head(20))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
9,5.568942,iwmogfm_4_0,299,512.0,0.0,500.0,3.0,polymnist_iwmogfm_2021_08_30_12_11_26_824957,0.922457,0.796820,0.289024,0.0.22-dev,https://ppb.hendrikklug.xyz/5c2a6676ea08.pdf,0.922457,NaN,0.547333,0.074786,0.540079,0.561758,0.140831,0.532492,0.535866,0.446089,0.061408,0.533035,0.138123,0.456854,0.057423,0.576795,0.068345,0.042195,0.564898,0.447889,0.048674,0.097341,0.444598,0.018003,0.000824,0.000928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,5.448727,iwmogfm_3_0,149,640.0,5.0,50.0,3.0,polymnist_iwmogfm_2021_09_11_18_03_52_050962,0.935102,0.788869,0.276762,0.0.22-dev,https://ppb.hendrikklug.xyz/a2c6ead436d7.pdf,0.935102,NaN,0.454753,0.067516,0.462112,0.651164,0.135219,0.453254,0.642939,0.373167,0.088208,0.449858,0.102885,0.383339,0.028826,0.670129,0.076952,0.034084,0.652379,0.378744,0.032455,0.095237,0.394985,0.011225,0.000989,0.001860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,5.446727,iwmogfm_4_0,149,640.0,2.0,50.0,3.0,polymnist_iwmogfm_2021_09_08_08_11_11_375546,0.939263,0.789883,0.275822,0.0.22-dev,https://ppb.hendrikklug.xyz/d3e2363d651c.pdf,0.939263,NaN,0.513547,0.067482,0.512007,0.573291,0.125469,0.494537,0.557229,0.410554,0.077702,0.506790,0.120756,0.411155,0.039329,0.593381,0.059914,0.033368,0.576218,0.403164,0.043463,0.081059,0.400548,0.014901,0.000602,0.003272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Method: MoPGfM
Mixture of Parameter Generalized f-means

In [15]:
display_df(df.loc[df['method'].str.startswith('mopgfm_')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
71,5.269037,mopgfm_1_1,999,1280.0,1.0,50.0,3.0,polymnist_mopgfm_2021_09_17_10_46_11_018977,0.900384,0.770166,0.266822,0.0.25-dev,https://ppb.hendrikklug.xyz/9936557336a9.pdf,0.900384,NaN,0.321365,0.179796,0.331181,0.458773,0.151867,0.330368,0.429396,0.368342,0.167194,0.316172,0.184188,0.388342,0.049299,0.433524,0.146764,0.048572,0.428156,0.377242,0.053116,0.167418,0.371361,0.149774,0.280401,0.271126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,5.261464,mopgfm_1_5,999,1280.0,1.0,50.0,3.0,polymnist_mopgfm_2021_09_17_10_46_19_832726,0.908174,0.784661,0.262391,0.0.25-dev,https://ppb.hendrikklug.xyz/76c7097387ea.pdf,0.908174,NaN,0.273350,0.195547,0.252379,0.480195,0.103226,0.260803,0.493049,0.402323,0.087048,0.247274,0.193603,0.412377,0.062391,0.469235,0.116950,0.046081,0.482565,0.413306,0.047519,0.179751,0.417643,0.140889,0.316380,0.203497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,5.170404,mopgfm_1_5,499,1280.0,2.0,50.0,2.0,polymnist_mopgfm_2021_09_23_10_58_10_284811,0.818234,0.635815,0.292340,0.0.27-dev,https://ppb.hendrikklug.xyz/590f3dbff9ab.pdf,0.818234,NaN,0.245624,0.310323,0.232002,0.471142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087739,0.456710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.161931,0.373246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Method: iwMopGfM
Mixture of Generalized f-means

In [16]:
display_df(df.loc[df['method'].str.startswith('iwmopgfm')][[*iwshow_cols]].sort_values(by=['score'], ascending=False).head(10))

compare_methods(df, methods = ['mopgfm_', 'iwmopgfm'], df_selectors= {'dataset': 'polymnist'}, show_cols=iwshow_cols)

,score,method,K,qz_x,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
106,5.159736,iwmopgfm__2_2,5.0,normal,149,512.0,2.0,50.0,3.0,polymnist_iwmopgfm__2021_09_26_21_44_20_810095,0.911995,0.775810,0.253318,0.0.28-dev,https://ppb.hendrikklug.xyz/a9cc52c35046.pdf,0.911995,NaN,0.254978,0.200559,0.259576,0.396443,0.240386,0.251986,0.404267,0.379828,0.158418,0.269886,0.265483,0.380325,0.071173,0.408720,0.141320,0.064981,0.412944,0.383000,0.068139,0.170082,0.375375,0.075503,0.228741,0.217517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,4.487042,iwmopgfm_2_2,1.0,normal,149,512.0,2.0,50.0,3.0,polymnist_iwmopgfm_2021_09_27_22_10_22_661556,0.860888,0.734084,0.200449,0.0.28-dev,https://ppb.hendrikklug.xyz/b76292626323.pdf,0.860888,NaN,0.154529,0.099230,0.164569,0.392274,0.143373,0.165152,0.372318,0.385581,0.074299,0.171317,0.148700,0.382468,0.029705,0.390778,0.079831,0.064729,0.390055,0.356812,0.028554,0.048962,0.384378,0.061185,0.158592,0.163392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,4.459507,iwmopgfm__2_2,5.0,laplace,149,512.0,2.0,50.0,3.0,polymnist_iwmopgfm__2021_09_26_21_44_20_811243,0.908205,0.765666,0.185645,0.0.28-dev,https://ppb.hendrikklug.xyz/4bb9976f21ef.pdf,0.908205,NaN,0.116927,0.130115,0.112585,0.334782,0.168825,0.113835,0.321900,0.339540,0.098257,0.113181,0.195943,0.349505,0.055631,0.324591,0.089572,0.060486,0.331149,0.326237,0.066784,0.130721,0.339208,0.048271,0.192787,0.0

,score,method,K,qz_x,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
71,5.269037,mopgfm_1_1,10.0,NaN,999,1280.0,1.0,50.0,3.0,polymnist_mopgfm_2021_09_17_10_46_11_018977,0.900384,0.770166,0.266822,0.0.25-dev,https://ppb.hendrikklug.xyz/9936557336a9.pdf,0.900384,NaN,0.321365,0.179796,0.331181,0.458773,0.151867,0.330368,0.429396,0.368342,0.167194,0.316172,0.184188,0.388342,0.049299,0.433524,0.146764,0.048572,0.428156,0.377242,0.053116,0.167418,0.371361,0.149774,0.280401,0.271126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,5.261464,mopgfm_1_5,10.0,NaN,999,1280.0,1.0,50.0,3.0,polymnist_mopgfm_2021_09_17_10_46_19_832726,0.908174,0.784661,0.262391,0.0.25-dev,https://ppb.hendrikklug.xyz/76c7097387ea.pdf,0.908174,NaN,0.273350,0.195547,0.252379,0.480195,0.103226,0.260803,0.493049,0.402323,0.087048,0.247274,0.193603,0.412377,0.062391,0.469235,0.116950,0.046081,0.482565,0.413306,0.047519,0.179751,0.417643,0.140889,0.316380,0.203497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,5.170404,mopgfm_1_5,10.0,NaN,499,1280.0,2.0,50.0,2.0,polymnist_mopgfm_2021_09_23_10_58_10_284811,0.818234,0.635815,0.292340,0.0.27-dev,https://ppb.hendrikklug.xyz/590f3dbff9ab.pdf,0.818234,NaN,0.245624,0.310323,0.232002,0.471142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.087739,0.456710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.161931,0.373246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

# Method: mopoe

In [17]:
display_df(df.loc[df['method'].str.startswith('mopoe')][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
39,6.453927,mopoe_4,149,640.0,2.0,50.0,3.0,MST_mopoe_2021_09_10_21_36_19_019375,0.926845,0.762611,0.383157,0.0.22-dev,NaN,0.926845,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.675000,0.015333,0.679465,0.477932,0.666566,0.450311,0.672088,0.002097,0.520672,0.297634,0.259556,0.495300,0.182475,0.464360,0.260525,0.011198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,6.370193,mopoe_4,299,1280.0,2.0,50.0,NaN,mnistsvhntext_mopoe_2021_09_23_10_57_27_193406,0.928698,0.752893,0.376303,0.0.27-dev,https://ppb.hendrikklug.xyz/816d1a565cd3.pdf,0.928698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.615525,0.042754,0.617729,0.388764,0.621300,0.370225,0.631144,0.022558,0.461493,0.252462,0.409895,0.405539,0.329378,0.418549,0.411885,0.021654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,4.905098,mopoe_4,299,1280.0,1.0,50.0,3.0,polymnist_mopoe_2021_09_11_09_02_21_115977,0.909447,0.755681,0.231854,0.0.22-dev,https://ppb.hendrikklug.xyz/3b0084ca2799.pdf,0.909447,NaN,0.214083,0.098175,0.185884,0.471787,0.115203,0.184143,0.468259,0.398936,0.116476,0.201889,0.093338,0.397145,0.024436,0.480910,0.101818,0.020434,0.484619,0.411297,0.024977,0.123608,0.396083,0.073741,0.263005,0.214244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

# Method: iwMoPoE

In [18]:
display_df(df.loc[df['method'].str.startswith('iwmopoe')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

compare_methods(df, methods = ['mopoe', 'iwmopoe'], df_selectors= {'dataset': 'polymnist'}, show_cols=iwshow_cols)


,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
11,4.978113,iwmopoe_4,299,512.0,2.5,0.0,3.0,polymnist_iwmopoe_2021_08_30_11_10_58_920798,0.927050,0.800829,0.228599,0.0.22-dev,https://ppb.hendrikklug.xyz/55a891e8a350.pdf,0.927050,NaN,0.153545,0.177231,0.145456,0.428791,0.194239,0.137761,0.427247,0.384391,0.155960,0.149623,0.236418,0.398489,0.046629,0.390047,0.172735,0.042732,0.432208,0.405857,0.044591,0.157653,0.431023,0.025578,0.171802,0.176381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,4.434592,iwmopoe_4,149,512.0,2.5,0.0,3.0,polymnist_iwmopoe_2021_09_26_11_03_56_497818,0.903512,0.712645,0.193419,0.0.28-dev,https://ppb.hendrikklug.xyz/41144a92b6f6.pdf,0.903512,NaN,0.215058,0.057633,0.223684,0.410654,0.071394,0.203279,0.399059,0.361592,0.064593,0.211303,0.075190,0.338362,0.075047,0.374251,0.062142,0.067705,0.406918,0.338833,0.073332,0.066840,0.392059,0.049934,0.051183,0.052013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,4.336127,iwmopoe_4,299,512.0,2.5,0.0,3.0,polymnist_iwmopoe_2021_08_23_16_40_35_951706,0.916581,0.774779,0.170533,0.0.22-dev,https://ppb.hendrikklug.xyz/d76c769df366.pdf,0.916581,NaN,0.113919,0.071774,0.115650,0.305773,0.132291,0.106304,0.290886,0.378460,0.108162,0.117825,0.108260,0.398861,0.037664,0.265829,0.105270,0.058077,0.298914,0.393579,0.047281,0.073497,0.421808,0.043964,0.049532,0.049214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

,score,method,K,qz_x,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
11,4.978113,iwmopoe_4,10.0,NaN,299,512.0,2.5,0.0,3.0,polymnist_iwmopoe_2021_08_30_11_10_58_920798,0.927050,0.800829,0.228599,0.0.22-dev,https://ppb.hendrikklug.xyz/55a891e8a350.pdf,0.927050,NaN,0.153545,0.177231,0.145456,0.428791,0.194239,0.137761,0.427247,0.384391,0.155960,0.149623,0.236418,0.398489,0.046629,0.390047,0.172735,0.042732,0.432208,0.405857,0.044591,0.157653,0.431023,0.025578,0.171802,0.176381,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,4.905098,mopoe_4,10.0,NaN,299,1280.0,1.0,50.0,3.0,polymnist_mopoe_2021_09_11_09_02_21_115977,0.909447,0.755681,0.231854,0.0.22-dev,https://ppb.hendrikklug.xyz/3b0084ca2799.pdf,0.909447,NaN,0.214083,0.098175,0.185884,0.471787,0.115203,0.184143,0.468259,0.398936,0.116476,0.201889,0.093338,0.397145,0.024436,0.480910,0.101818,0.020434,0.484619,0.411297,0.024977,0.123608,0.396083,0.073741,0.263005,0.214244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,4.701858,mopoe_4,10.0,laplace,149,512.0,0.0,0.0,3.0,polymnist_mopoe_2021_09_28_11_50_57_033739,0.851537,0.766592,0.217267,0.0.28-dev,https://ppb.hendrikklug.xyz/243709569c7f.pdf,0.851537,NaN,0.420831,0.010231,0.422262,0.418539,0.006587,0.426107,0.425470,0.428502,0.011700,0.432242,0.008501,0.441479,0.006170,0.419322,0.008273,0.008410,0.428291,0.444648,0.005471,0.003019,0.410886,0.017695,0.002309,0.007466,NaN,NaN,NaN,NaN,NaN,NaN,

### base params

In [19]:
display_df(df.loc[(df['method'].str.startswith('joint_elbo')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: PGfM
Parameter Generalized f-means

In [20]:
display_df(df.loc[df['method'].str.startswith('pgfm_')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: GfM
Generalized f-means

In [21]:
display_df(df.loc[df['method'].str.startswith('gfm_')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### GfM vs EGfM

In [22]:
display_df(df.loc[(df['method'].str.startswith('gfm_')) | df['method'].str.startswith('egfm')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### GfM vs joint_elbo

In [23]:
display_df(df.loc[((df['method'].str.startswith('joint_elbo')) | (df['method'].str.startswith('gfm_')))  & (
        df['end_epoch'].astype(int) == 99) & (df['beta'] == 1) & (df['amortized_flow'] == 0) & (
                          df['class_dim'] == 256) & (df['weighted_mixture'] == 0) & (
                          df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(
    by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: GfMoP

In [24]:
display_df(df.loc[df['method'].str.startswith('gfmop')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### GfM vs GfMoP

In [25]:
display_df(df.loc[((df['method'].str.startswith('gfm_')) | (df['method'].str.startswith('gfmop')))  & (
        df['end_epoch'].astype(int) == 99) & (df['beta'] == 1) & (df['amortized_flow'] == 0) & (
                          df['class_dim'] == 256) & (df['weighted_mixture'] == 0) & (
                          df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(
    by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: MoFoPoE

In [26]:
display_df(df.loc[df['method'].str.startswith('mofop')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### mofop vs joint_elbo: 0 flows, 1 mod

In [27]:
display_df(df.loc[((df['method'].str.startswith('mofop')) | (df['method'].str.startswith('joint_elbo')))  &
        ((df['num_flows'].astype(int) == 0) | (df['num_flows'].astype(int) == 5)) & (df['num_mods'] == 1)][[*show_cols]].sort_values(
    by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: FoMFoP
flow of mixture of flow of product of experts

In [28]:
display_df(df.loc[df['method'].str.startswith('fomfop')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### Comparison with joint_elbo (moepoe)

In [29]:
display_df(df.loc[(df['method'].str.startswith('fomfop')) | (df['method'].str.startswith('joint_elbo'))][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
2,4.527812,joint_elbo,299,512.0,2.5,0.0,3.0,polymnist_joint_elbo_2021_08_20_22_57_41_889181,0.894959,0.745343,0.197937,0.0.21-dev,NaN,0.894959,NaN,0.221744,0.034559,0.218787,0.382788,0.01775,0.227894,0.385091,0.315827,0.033122,0.225078,0.025718,0.324317,0.100872,0.367341,0.032096,0.103953,0.37401,0.301454,0.108965,0.031325,0.326822,0.162662,0.230781,0.197521,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Method: Planar Mixture

In [30]:
display_df(df.loc[df['method'].str.startswith('planar_mixture')][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### Planar Mixture vs MoE

In [31]:
display_base_params(df,methods = ['planar_mixture', 'moe'], show_cols=show_cols)

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### More than 1 flow, 3 mods

In [32]:
display_df(df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_flows'])  & (df['num_mods'] > 1)][[*show_cols]].sort_values(by=['score'], ascending=False).head(3))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### without flows
looks good without flows.

In [33]:
display_df(df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_flows'] == 0) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### 1 mod

In [34]:
display_df(df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_mods'] == 1) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


## Without amortized flows

In [35]:
display_df(df.loc[(df['method'].str.startswith('planar_mixture'))&(df['amortized_flow'] == 0.0)][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### base params

In [36]:
display_df(df.loc[(df['method'].str.startswith('planar_mixture')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: pfom


In [37]:
display_df(df.loc[df['method'].str.startswith('pfom')][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


## Without amortized flows

In [38]:
display_df(df.loc[(df['method'].str.startswith('pfom'))&(df['amortized_flow'] == 0.0)][[*show_cols]].sort_values(by=['score'], ascending=False).head(10))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### without flows

In [39]:
display_df(df.loc[(df['method'].str.startswith('pfom')) & (df['num_flows'] == 0) ][[*show_cols]].sort_values(by=['score'], ascending=False).head(3))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### base params

In [40]:
display_df(df.loc[(df['method'].str.startswith('pfom')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### pfom vs moe

In [41]:
display_base_params(df,methods = ['pfom', 'moe'], show_cols=show_cols)

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Runs with current version

In [42]:
version = BaseFlagsSetup.get_version_from_setup_config()
display_df(df.loc[df['version'] == version][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA
106,5.159736,iwmopgfm__2_2,149,512.0,2.0000,50.0,3.0,polymnist_iwmopgfm__2021_09_26_21_44_20_810095,0.911995,0.775810,0.253318,0.0.28-dev,https://ppb.hendrikklug.xyz/a9cc52c35046.pdf,0.911995,NaN,0.254978,0.200559,0.259576,0.396443,0.240386,0.251986,0.404267,0.379828,0.158418,0.269886,0.265483,0.380325,0.071173,0.408720,0.141320,0.064981,0.412944,0.383000,0.068139,0.170082,0.375375,0.075503,0.228741,0.217517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,5.091045,iwmogfm_4_2,149,640.0,0.0000,0.0,3.0,polymnist_iwmogfm_2021_09_26_13_53_43_584251,0.860654,0.800163,0.248866,0.0.28-dev,https://ppb.hendrikklug.xyz/eade4b77d40e.pdf,0.860654,NaN,0.433759,0.027916,0.434287,0.536216,0.105993,0.422132,0.529035,0.390667,0.043318,0.444179,0.151427,0.396887,0.023240,0.530468,0.043345,0.068071,0.523103,0.399043,0.023192,0.027879,0.410631,0.006585,0.000156,0.001268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,4.828581,iwmogfm_amortized_4_2,149,640.0,0.0000,50.0,3.0,polymnist_iwmogfm_amortized_2021_09_27_08_25_38_189266,0.909889,0.790662,0.217783,0.0.28-dev,https://ppb.hendrikklug.xyz/1dcf9e3952b0.pdf,0.909889,NaN,0.402111,0.026655,0.395184,0.457488,0.081521,0.400195,0.427779,0.374249,0.056394,0.399842,0.056180,0.376384,0.011799,0.481915,0.040835,0.011010,0.439438,0.367544,0.010951,0.024927,0.382751,0.000614,0.000167,0.000853,NaN,NaN,NaN,Na

# Method: fomop
Flow of mixture of product of experts

In [43]:
display_df(df.loc[df['method'].str.startswith('fomop')][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### fomop vs fomfop vs joint_elbo

In [44]:
display_base_params(df,methods = ['fomop', 'fomfop', 'joint_elbo'], show_cols=show_cols)

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: fomfop
Flow of mixture of flow of product of experts

In [45]:
display_df(df.loc[df['method'].str.startswith('fomfop')][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Runs with more than 1000 epochs

In [46]:
display_df(df.loc[df['end_epoch'].astype(int) > 1000][[*show_cols]].sort_values(by=['score'], ascending=False))



,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: poe

In [47]:
display_df(df.loc[df['method'].str.startswith('poe')][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### base params

In [48]:
display_df(df.loc[(df['method'].str.startswith('poe')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


### poe vs pope

In [49]:
display_base_params(df,methods = ['pope', 'poe'], show_cols=show_cols)

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Method: moe

### base params

In [50]:
display_df(df.loc[(df['method'].str.startswith('moe')) & (df['num_flows'] == 5) & (df['end_epoch'].astype(int) == 99)& (df['beta'] == 1) & (df['amortized_flow'] == 0) & (df['class_dim'] == 256) & (df['weighted_mixture'] == 0)& (df['initial_learning_rate'].astype(float) == 0.0005)][[*show_cols]].sort_values(by=['score'], ascending=False))

,score,method,end_epoch,class_dim,max_beta,beta_warmup,num_mods,_id,score_lr,score_gen,score_prd,version,expvis_url,score_lr_q0,score_lr_zk,prd_score_m0_m0,prd_score_m0_m1,prd_score_m0_m1_m0,prd_score_m0_m1_m1,prd_score_m0_m1_m2,prd_score_m0_m1_m2_m0,prd_score_m0_m1_m2_m1,prd_score_m0_m1_m2_m2,prd_score_m0_m2,prd_score_m0_m2_m0,prd_score_m0_m2_m1,prd_score_m0_m2_m2,prd_score_m1_m0,prd_score_m1_m1,prd_score_m1_m2,prd_score_m1_m2_m0,prd_score_m1_m2_m1,prd_score_m1_m2_m2,prd_score_m2_m0,prd_score_m2_m1,prd_score_m2_m2,prd_score_random_m0,prd_score_random_m1,prd_score_random_m2,prd_score_m0_m1_m2_m3,prd_score_m0_m1_m2_m3_m0,prd_score_m0_m1_m2_m3_m1,prd_score_m0_m1_m2_m3_m2,prd_score_m0_m1_m2_m3_m3,prd_score_m0_m1_m3,prd_score_m0_m1_m3_m0,prd_score_m0_m1_m3_m1,prd_score_m0_m1_m3_m2,prd_score_m0_m1_m3_m3,prd_score_m0_m2_m3,prd_score_m0_m2_m3_m0,prd_score_m0_m2_m3_m1,prd_score_m0_m2_m3_m2,prd_score_m0_m2_m3_m3,prd_score_m0_m3,prd_score_m0_m3_m0,prd_score_m0_m3_m1,prd_score_m0_m3_m2,prd_score_m0_m3_m3,prd_score_m1_m2_m3,prd_score_m1_m2_m3_m0,prd_score_m1_m2_m3_m1,prd_score_m1_m2_m3_m2,prd_score_m1_m2_m3_m3,prd_score_m1_m3,prd_score_m1_m3_m0,prd_score_m1_m3_m1,prd_score_m1_m3_m2,prd_score_m1_m3_m3,prd_score_m2_m3,prd_score_m2_m3_m0,prd_score_m2_m3_m1,prd_score_m2_m3_m2,prd_score_m2_m3_m3,prd_score_m3_m0,prd_score_m3_m1,prd_score_m3_m2,prd_score_m3_m3,prd_score_random_m3,prd_score_mnist_mnist,prd_score_mnist_svhn,prd_score_mnist_svhn_mnist,prd_score_mnist_svhn_svhn,prd_score_mnist_svhn_text_mnist,prd_score_mnist_svhn_text_svhn,prd_score_mnist_text_mnist,prd_score_mnist_text_svhn,prd_score_random_mnist,prd_score_random_svhn,prd_score_svhn_mnist,prd_score_svhn_svhn,prd_score_svhn_text_mnist,prd_score_svhn_text_svhn,prd_score_text_mnist,prd_score_text_svhn,prd_score_Lateral_Lateral,prd_score_Lateral_PA,prd_score_Lateral_PA_Lateral,prd_score_Lateral_PA_PA,prd_score_Lateral_PA_text_Lateral,prd_score_Lateral_PA_text_PA,prd_score_Lateral_text_Lateral,prd_score_Lateral_text_PA,prd_score_PA_Lateral,prd_score_PA_PA,prd_score_PA_text_Lateral,prd_score_PA_text_PA,prd_score_random_Lateral,prd_score_random_PA,prd_score_text_Lateral,prd_score_text_PA


# Generate samples with the model that has the highest score.

In [51]:
best_score_id = df[df.score == df.score.max()]._id.item()

print(best_score_id)






MST_mopoe_2021_09_10_21_36_19_019375


In [52]:
show_generated_figs(_id = best_score_id)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Hendrik/src/MMVAE_Hub/configs/mst/local_config.json'